In [1]:
# Parameters
N_Trials = 2


### Init s3 bucket for project
The purpose of this notebook is to create an s3 bucket for our group project, then upload the hospital data there. We should then create an Athena table and test the data.

In [2]:
import pandas as pd

df = pd.read_csv("data/hospital_readmissions.csv")
display(df.head(10))

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,[70-80),8,72,1,18,2,0,0,Missing,Circulatory,Respiratory,Other,no,no,no,yes,no
1,[70-80),3,34,2,13,0,0,0,Other,Other,Other,Other,no,no,no,yes,no
2,[50-60),5,45,0,18,0,0,0,Missing,Circulatory,Circulatory,Circulatory,no,no,yes,yes,yes
3,[70-80),2,36,0,12,1,0,0,Missing,Circulatory,Other,Diabetes,no,no,yes,yes,yes
4,[60-70),1,42,0,7,0,0,0,InternalMedicine,Other,Circulatory,Respiratory,no,no,no,yes,no
5,[40-50),2,51,0,10,0,0,0,Missing,Other,Other,Other,no,no,no,no,yes
6,[50-60),4,44,2,21,0,0,0,Missing,Injury,Other,Other,no,normal,yes,yes,no
7,[60-70),1,19,6,16,0,0,1,Other,Circulatory,Other,Other,no,no,no,yes,yes
8,[80-90),4,67,3,13,0,0,0,InternalMedicine,Digestive,Other,Other,no,no,no,no,yes
9,[70-80),8,37,1,18,0,0,0,Family/GeneralPractice,Respiratory,Respiratory,Other,no,no,yes,yes,no


In [3]:
# Preprocess category data in age, medical_specialty, and diagnoses into int.
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['glucose_test', 'A1Ctest', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'change', 'diabetes_med', 'readmitted']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = {index: label for index, label in enumerate(le.classes_)}  # Save mappings

# Display the processed data
print("\nProcessed Data:")
display(df.head())

print("Columns before NaN drop: ", len(df))
# Drop nan?
df = df.dropna()
print("Columns after NaN drop: ", len(df))


Processed Data:


,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,3,8,72,1,18,2,0,0,4,0,7,6,1,1,0,1,0
1,3,3,34,2,13,0,0,0,5,6,6,6,1,1,0,1,0
2,1,5,45,0,18,0,0,0,4,0,0,0,1,1,1,1,1
3,3,2,36,0,12,1,0,0,4,0,6,1,1,1,1,1,1
4,2,1,42,0,7,0,0,0,3,6,0,7,1,1,0,1,0


Columns before NaN drop:  25000
Columns after NaN drop:  25000


In [4]:
print(label_encoders)

encodings_file = "encodings.json"

{'glucose_test': {0: 'high', 1: 'no', 2: 'normal'}, 'A1Ctest': {0: 'high', 1: 'no', 2: 'normal'}, 'age': {0: '[40-50)', 1: '[50-60)', 2: '[60-70)', 3: '[70-80)', 4: '[80-90)', 5: '[90-100)'}, 'medical_specialty': {0: 'Cardiology', 1: 'Emergency/Trauma', 2: 'Family/GeneralPractice', 3: 'InternalMedicine', 4: 'Missing', 5: 'Other', 6: 'Surgery'}, 'diag_1': {0: 'Circulatory', 1: 'Diabetes', 2: 'Digestive', 3: 'Injury', 4: 'Missing', 5: 'Musculoskeletal', 6: 'Other', 7: 'Respiratory'}, 'diag_2': {0: 'Circulatory', 1: 'Diabetes', 2: 'Digestive', 3: 'Injury', 4: 'Missing', 5: 'Musculoskeletal', 6: 'Other', 7: 'Respiratory'}, 'diag_3': {0: 'Circulatory', 1: 'Diabetes', 2: 'Digestive', 3: 'Injury', 4: 'Missing', 5: 'Musculoskeletal', 6: 'Other', 7: 'Respiratory'}, 'change': {0: 'no', 1: 'yes'}, 'diabetes_med': {0: 'no', 1: 'yes'}, 'readmitted': {0: 'no', 1: 'yes'}}


In [5]:
# Initialize s3 bucket for our group
import boto3
bucket_name = "group3-project-bucket"
region = "us-east-1"
s3_client = boto3.client("s3", region_name=region)

# Create the bucket
s3_client.create_bucket(
    Bucket=bucket_name
)

print(f"Bucket '{bucket_name}' created successfully!")

Bucket 'group3-project-bucket' created successfully!


In [6]:
file_path = f"s3://{bucket_name}/parquet/hospital_data.parquet"
df.to_parquet(file_path, index=False)

In [7]:
!aws s3 ls $bucket_name/parquet/

2025-03-01 23:35:15     169631 hospital_data.parquet


In [8]:
import json
# Add label encodings json into s3 bucket.
# Save the label encodings as JSON

encodings_file = "encodings.json"
with open(encodings_file, "w") as f:
    json.dump(label_encoders, f)

encodings_key = "encodings/encodings.json"
s3_client.upload_file(encodings_file, bucket_name, encodings_key)
print(f"Encodings saved and uploaded to s3://{bucket_name}/{encodings_key}")

Encodings saved and uploaded to s3://group3-project-bucket/encodings/encodings.json


In [9]:
!aws s3 ls $bucket_name/encodings/

2025-03-01 23:35:16        967 encodings.json


## Create Athena Tables

In [10]:
# Programmically generate a schmema for the athena engine
pandas_to_athena_types = {
    "object": "STRING",
    "int64": "BIGINT",
    "float64": "DOUBLE",
    "datetime64[ns]": "TIMESTAMP",
    "bool": "BOOLEAN",
}

schema = []
for col_name, dtype in df.dtypes.items():
    athena_type = pandas_to_athena_types.get(str(dtype), "STRING")
    schema.append(f"{col_name} {athena_type}")

print(schema)

['age BIGINT', 'time_in_hospital BIGINT', 'n_lab_procedures BIGINT', 'n_procedures BIGINT', 'n_medications BIGINT', 'n_outpatient BIGINT', 'n_inpatient BIGINT', 'n_emergency BIGINT', 'medical_specialty BIGINT', 'diag_1 BIGINT', 'diag_2 BIGINT', 'diag_3 BIGINT', 'glucose_test BIGINT', 'A1Ctest BIGINT', 'change BIGINT', 'diabetes_med BIGINT', 'readmitted BIGINT']


In [11]:
from pyathena import connect

database_name = "group_project_db"
table_name = "hospital_readmissions"

connection = connect(
    s3_staging_dir = f"s3://{bucket_name}/athena-results/",
    region_name="us-east-1",
)

create_database_query = f"CREATE DATABASE IF NOT EXISTS {database_name}"
connection.cursor().execute(create_database_query)
print(f"Database {database_name} created or already exists")

Database group_project_db created or already exists


In [12]:
schema_str = ",\n    ".join(schema)

drop_query = f"""
DROP TABLE IF EXISTS {database_name}.{table_name};
"""

create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    {schema_str}
)
STORED AS PARQUET
LOCATION 's3://{bucket_name}/parquet/';
"""

print(drop_query)
print(create_table_query)


DROP TABLE IF EXISTS group_project_db.hospital_readmissions;


CREATE EXTERNAL TABLE IF NOT EXISTS group_project_db.hospital_readmissions (
    age BIGINT,
    time_in_hospital BIGINT,
    n_lab_procedures BIGINT,
    n_procedures BIGINT,
    n_medications BIGINT,
    n_outpatient BIGINT,
    n_inpatient BIGINT,
    n_emergency BIGINT,
    medical_specialty BIGINT,
    diag_1 BIGINT,
    diag_2 BIGINT,
    diag_3 BIGINT,
    glucose_test BIGINT,
    A1Ctest BIGINT,
    change BIGINT,
    diabetes_med BIGINT,
    readmitted BIGINT
)
STORED AS PARQUET
LOCATION 's3://group3-project-bucket/parquet/';



In [13]:
# Execute the query to create the table
connection.cursor().execute(drop_query)
print(f"Table '{table_name}' dropped if exists in database '{database_name}'.")

# Execute the query to create the table
connection.cursor().execute(create_table_query)
print(f"Table '{table_name}' created successfully in database '{database_name}'.")

Table 'hospital_readmissions' dropped if exists in database 'group_project_db'.


Table 'hospital_readmissions' created successfully in database 'group_project_db'.


In [14]:
def run_athena_query(query, output_s3_path, region="us-east-1"):
    with connect(
        s3_staging_dir=output_s3_path,
        region_name=region,
    ) as conn:
        df = pd.read_sql(query, conn)
        
    return df

In [15]:
query = f'''
SELECT * 
FROM {database_name}.{table_name} 
LIMIT 10
'''

query_df = run_athena_query(query, f"s3://{bucket_name}/athena-results/")
display(query_df)

/tmp/ipykernel_6824/3350967399.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,a1ctest,change,diabetes_med,readmitted
0,3,8,72,1,18,2,0,0,4,0,7,6,1,1,0,1,0
1,3,3,34,2,13,0,0,0,5,6,6,6,1,1,0,1,0
2,1,5,45,0,18,0,0,0,4,0,0,0,1,1,1,1,1
3,3,2,36,0,12,1,0,0,4,0,6,1,1,1,1,1,1
4,2,1,42,0,7,0,0,0,3,6,0,7,1,1,0,1,0
5,0,2,51,0,10,0,0,0,4,6,6,6,1,1,0,0,1
6,1,4,44,2,21,0,0,0,4,3,6,6,1,2,1,1,0
7,2,1,19,6,16,0,0,1,5,0,6,6,1,1,0,1,1
8,4,4,67,3,13,0,0,0,3,2,6,6,1,1,0,0,1
9,3,8,37,1,18,0,0,0,2,7,7,6,1,1,1,1,0
